In [13]:
import numpy as np
from scipy.io import loadmat
import scipy
from scipy import signal
from matplotlib import pyplot  as plt
from scipy.fftpack import fft
import PyQt5
import peakutils
import timeit
import pandas as pd
from IPython.display import display, HTML
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import svm , datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsOneClassifier
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
import sklearn
%matplotlib qt5

In [14]:
class plot ():
    def __init__(self, sample_freq):
        self.S = sample_freq
    def time(self, data, label):
        N = len(data)
        T = 1.0 / self.S
        t = np.linspace(0 , N*T , N )
        plt.figure(figsize=(9,3))
        plt.plot(t, data, color= 'm')
        plt.grid()
        plt.ylabel('Amplitude(mu Volt)')
        plt.xlabel('Time(Sec)')
        plt.title(label)
        plt.tight_layout()
        plt.show()
    def freq(self, data , label):
        #plot FFT of signal
        N = len(data)
        T = 1.0 / self.S
        xf = np.linspace(0.0, 1.0/(2.0*T), N/2)         
        fig, ax = plt.subplots(2, 1, figsize=(7, 4))
        ax[0].plot(xf, 2.0/N * np.abs(data[:N//2]).T, color='blue')
        ax[0].set_ylabel("Amplitude ", color='blue')
        ax[0].grid()
        ax[1].plot(xf, np.unwrap(np.angle(data[:N//2]).T)*180/np.pi, color='green')
        ax[1].set_ylabel("Angle (degrees)", color='green')
        ax[1].set_xlabel("Frequency (Hz)")
        ax[1].grid()
        plt.suptitle(label)
        plt.show()
        
    def filt(self, b, a ,label):
        w, h = signal.freqz(b, a)
        freq = w*self.S/(2*np.pi)
        fig, ax = plt.subplots(2, 1, figsize=(7, 4))
        ax[0].plot(freq, (abs(h)), color='blue')
        ax[0].set_title("Frequency Response")
        ax[0].set_ylabel("Amplitude ", color='blue')
        ax[0].grid()
        ax[1].plot(freq, np.unwrap(np.angle(h))*180/np.pi, color='green')
        ax[1].set_ylabel("Angle (degrees)", color='green')
        ax[1].set_xlabel("Frequency (Hz)")
        ax[1].grid()
        plt.suptitle(label)
        plt.show()
    def peak(self ,data ,  indices , label ):
        self.time(data, label)
        for i in range(len(indices)):
            plt.plot(indices[i]/self.S,data[indices[i]] , '*' , color='r')
            plt.show()
        

In [3]:
normal1 = np.array(loadmat('Normal1.mat')['val'])[1]
normal2 = np.array(loadmat('Normal2.mat')['val'])[1]
normal3 = np.array(loadmat('Normal3.mat')['val'])[1]
normal4 = np.array(loadmat('Normal4.mat')['val'])[2]
normal5 = np.array(loadmat('Normal5.mat')['val'])[1]
normal = [normal1,normal2 , normal3 , normal4 , normal5]

# preprocessing

### ECG in time and freq domain

In [15]:
mother_signal = np.zeros_like(normal)
HRVmother_org = np.zeros(len(normal))
for n in range (len(normal)):
    mt1 = normal[n]
    fs = 1000
    N = len(mt1)
    mother = plot(fs)
#     mother.time(mt1,'Initial Mother ECG signal in time domain')

    # Notch filter
    f0 = 50.0  # Frequency to be removed from signal (Hz)
    w0 = f0/(fs/2)  # Normalized Frequency
    b50, a50 = scipy.signal.butter(2, [w0-.02, w0+.02], 'stop')
    mt2 = scipy.signal.filtfilt(b50, a50, mt1)

    # low pass filter
    w0 = 50/(fs/2)  
    b125, a125 = scipy.signal.butter(8, w0, 'low')
    mt3 = scipy.signal.filtfilt(b125, a125, mt2)
    mother_signal[n]= scipy.signal.filtfilt(b125, a125, mt2)

#     mother.time(mother_signal[n], 'Denoisde signal')

    indices = peakutils.indexes(mother_signal[n], thres=0.97, min_dist=600)
#     mother.peak(mother_signal[n], indices , 'peak detected on original mother-ECG ')
    hr=0
    HRVmother_org[n]=0
    for i in range(len(indices)-1):
        if (indices[i+1] - indices[i] > fs/ 2) and (indices[i+1] - indices[i] < 1.7*fs) :
            HRVmother_org[n] += indices[i+1] - indices[i]
            hr += 1
    HRVmother_org[n] =( 1/(HRVmother_org[n] / hr / fs)) * 60
    print ('Mother heart rate calculated from original mother_ECG : %.2f '% HRVmother_org[n])
mother.time(mt1,'Initial Mother ECG signal in time domain')
mother.time(mother_signal[n], 'Denoisde signal')
mother.peak(mother_signal[n], indices , 'peak detected on original mother-ECG ')

C:\Users\Mohamad\Anaconda3\lib\site-packages\peakutils\peak.py:37: RuntimeWarning: overflow encountered in short_scalars
  thres = thres * (np.max(y) - np.min(y)) + np.min(y)


Mother heart rate calculated from original mother_ECG : 74.14 
Mother heart rate calculated from original mother_ECG : 88.43 
Mother heart rate calculated from original mother_ECG : 71.21 
Mother heart rate calculated from original mother_ECG : 73.04 
Mother heart rate calculated from original mother_ECG : 61.60 


# Generate infant's ECG

In [17]:
it = {}
mix_ideal={}
HRVinfant_org = np.zeros(len(normal))
resmpl = np.zeros(len(normal))
for n in range ((len(normal))):
    resmpl[n] = int(N// (10 /np.random.randint(5,9)))

    it[n]= signal.resample(x=mother_signal[n] , num=resmpl[n])
    st = N - resmpl[n] - 1
    st = np.random.randint(0,st)
    mother_signal_new= mother_signal[n][st:st+resmpl[n]]


    mix_ideal[n] = (np.random.random()/3.5 +.1)*it[n] + mother_signal_new
#     mother.time(mix_ideal[n] , 'Mixed ideal ECG in time domian')

    indices = peakutils.indexes(it[n], thres=0.81, min_dist=300)
#     mother.peak(it[n], indices , 'peak detected on original infant-ECG ')
    hr=0
    HRVinfant_org[n] =0
    for i in range(len(indices)-1):
        if (indices[i+1] - indices[i] > fs/ 3) and (indices[i+1] - indices[i] < fs) :
            HRVinfant_org[n] += indices[i+1] - indices[i]
            hr += 1
    HRVinfant_org[n] =( 1/(HRVinfant_org[n] / hr / fs)) * 60
    print ('Infant heart rate calculated from original infant_ECG : %.2f '% HRVinfant_org[n])
mother.peak(it[n], indices , 'peak detected on original infant-ECG ')


C:\Users\Mohamad\Anaconda3\lib\site-packages\scipy\signal\signaltools.py:1889: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y = zeros(newshape, 'D')
C:\Users\Mohamad\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Infant heart rate calculated from original infant_ECG : 123.58 
Infant heart rate calculated from original infant_ECG : 126.35 
Infant heart rate calculated from original infant_ECG : 89.02 
Infant heart rate calculated from original infant_ECG : 142.25 
Infant heart rate calculated from original infant_ECG : 97.15 


# Extract mom's and baby's heart rate from mixed data

In [21]:
HRVmother = np.zeros(len(normal))
for n in range ((len(normal))):

    indicesm = peakutils.indexes(mix_ideal[n], thres=0.65, min_dist=600)
#     mother.peak(mix_ideal[n], indicesm , 'peak detected on denoised mix-ECG ')
    hr=0
    HRVmother[n] = 0
    for i in range(len(indicesm)-1):
        if (indicesm[i+1] - indicesm[i] > fs/ 2) and (indicesm[i+1] - indicesm[i] < 1.7*fs) :
            HRVmother[n] += indicesm[i+1] - indicesm[i]
            hr += 1
    HRVmother[n] =( 1/(HRVmother[n] / hr / fs)) * 60
    print ('Mother heart rate calculated from mix_ECG : %.2f '% HRVmother[n])
mother.peak(mix_ideal[n], indicesm , 'peak detected on denoised mix-ECG ')


Mother heart rate calculated from mix_ECG : 74.67 
Mother heart rate calculated from mix_ECG : 88.99 
Mother heart rate calculated from mix_ECG : 71.55 
Mother heart rate calculated from mix_ECG : 72.67 
Mother heart rate calculated from mix_ECG : 58.22 


# remove mother signal based on correlation form mix-ecg

In [20]:
HRVinfant = np.zeros(len(normal))
babe_ECG ={}
# for n in range ((len(normal))):

for n in range (5):

    corr = signal.correlate(mother_signal[n],mix_ideal[n]  , 'same')
    max_corr = np.argmax(corr)

    fig, (ax_mom,ax_mixx, ax_corr) = plt.subplots(3, 1, sharex=True)
    ax_mom.plot(mother_signal[n])
    ax_mom.set_title('Mother signal'+ str(n))

    ax_mixx.plot((np.hstack((np.zeros(max_corr - resmpl[n] //2),mix_ideal[n]))))
    ax_mixx.plot((max_corr- resmpl[n] //2),0, '<' , color='r')
    ax_mixx.plot((max_corr+ resmpl[n] //2),0, '>', color= 'r')
    ax_mixx.set_title('Mix siganl'+ str(n))

    ax_corr.plot(corr)
    ax_corr.plot(max_corr, corr[max_corr], 'ro')
    ax_corr.plot((max_corr- resmpl[n]//2),0, '<' , color='r')
    ax_corr.plot((max_corr+ resmpl[n]//2),0, '>', color= 'r')
    ax_corr.set_title('Cross-correlated with Mother and mixed signal'+ str(n))
    ax_mom.margins(0, 0.1)

    fig.tight_layout()
    fig.show()

    babe_ECG[n] = mix_ideal[n][:(resmpl[n]//2)+(resmpl[n]//2)] - mother_signal[n][max_corr - resmpl[n]//2 : max_corr + resmpl[n]//2]
    indicesb = peakutils.indexes(babe_ECG[n], thres=0.65, min_dist=300)
#     mother.peak(babe_ECG[n], indicesb , 'peak detected on extracted baby ECG ')
    hr=0
    HRVinfant[n] = 0
    for i in range(len(indicesb)-1):
        if (indicesb[i+1] - indicesb[i] > fs/ 3) and (indicesb[i+1] - indicesb[i] < fs) :
            HRVinfant[n] += indicesb[i+1] - indicesb[i]
            hr += 1
    HRVinfant[n] =( 1/(HRVinfant[n] / hr / fs)) * 60
    print ('infant heart rate calculated from mix_ECG : %.2f '% HRVinfant[n])
mother.peak(babe_ECG[n], indicesb , 'peak detected on extracted baby ECG ')

C:\Users\Mohamad\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\Mohamad\Anaconda3\lib\site-packages\ipykernel\__main__.py:29: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


infant heart rate calculated from mix_ECG : 123.58 
infant heart rate calculated from mix_ECG : 126.35 
infant heart rate calculated from mix_ECG : 89.02 
infant heart rate calculated from mix_ECG : 142.25 
infant heart rate calculated from mix_ECG : 97.15 


# result

In [22]:
for n in range(len(normal)):
    print ('Mother original heart rate calculated from mother_ECG : %.2f '% HRVmother_org[n])
    print ('Infant original heart rate calculated from mother_ECG : %.2f '% HRVinfant_org[n])
    print ('Mother heart rate calculated from mix_ECG : %.2f '% HRVmother[n])
    print ('Infant heart rate calculated from mix_ECG : %.2f '% HRVinfant[n])

Mother original heart rate calculated from mother_ECG : 74.14 
Infant original heart rate calculated from mother_ECG : 123.58 
Mother heart rate calculated from mix_ECG : 74.67 
Infant heart rate calculated from mix_ECG : 123.58 
Mother original heart rate calculated from mother_ECG : 88.43 
Infant original heart rate calculated from mother_ECG : 126.35 
Mother heart rate calculated from mix_ECG : 88.99 
Infant heart rate calculated from mix_ECG : 126.35 
Mother original heart rate calculated from mother_ECG : 71.21 
Infant original heart rate calculated from mother_ECG : 89.02 
Mother heart rate calculated from mix_ECG : 71.55 
Infant heart rate calculated from mix_ECG : 89.02 
Mother original heart rate calculated from mother_ECG : 73.04 
Infant original heart rate calculated from mother_ECG : 142.25 
Mother heart rate calculated from mix_ECG : 72.67 
Infant heart rate calculated from mix_ECG : 142.25 
Mother original heart rate calculated from mother_ECG : 61.60 
Infant original hea

In [ ]:
# plt.close('all')

In [23]:
# single= np.zeros_like(mother_signal)
# for n in range(len(normal)):
# #     for i in range (len(normal)):
#     n1 = np.linspace(0, N/fs, N)
#     noise = ((1/6)*np.cos(1*np.pi*2*n1) +(1/2)*np.sin(.3*np.pi*2*n1))
#     single[n] = 2000*noise + mother_signal[n]
# #     single[n] = 10000*noise
# # mother.time(single[0], 'Non pregnant ECG with simulated arrhythmia')
# mother.time(noise, 'Non pregnant ECG with simulated arrhythmia')

single= np.zeros_like(mother_signal)
for n in range(len(normal)):
#     for i in range (len(normal)):
    n1 = np.linspace(0, N/fs, N)
    w0 = 5/(1000/2) 
    b2, a2 = scipy.signal.butter(5, w0, 'high')
#     mother.filt(b2,a2,'High pass filter to remove motion artifact')

    noise = scipy.signal.filtfilt(b2, a2, mother_signal[n])
    single[n] = .13*noise + mother_signal[n]
mother.time(single[0], 'Non pregnant ECG with simulated arrhythmia')
mother.time(noise, 'arrhythmia noise')


C:\Users\Mohamad\Anaconda3\lib\site-packages\matplotlib\pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [24]:
for pp in range (3):
    DATA={}
    for n in range(len(normal)):
        if pp == 0:
            DATA[n]=mother_signal[n], peakutils.indexes(mother_signal[n], thres=0.97, min_dist=600) , 0
            DATA[n+5]=mix_ideal[n],peakutils.indexes(mix_ideal[n], thres=0.65, min_dist=600) ,1
            labell = 'class0: Non pregnant healthy person   class1: pregnant person healthy person'
        if pp == 1:
            DATA[n]=single[n], peakutils.indexes(single[n], thres=0.97, min_dist=600) , 0
            DATA[n+5]=mix_ideal[n],peakutils.indexes(mix_ideal[n], thres=0.65, min_dist=600) ,1
            labell = 'class0: other arrhythmia   class1: pregnant healthy person'
        if pp == 2:
            DATA[n]=single[n], peakutils.indexes(single[n], thres=0.97, min_dist=600) , 0
            DATA[n+5]=mix_ideal[n],peakutils.indexes(mix_ideal[n], thres=0.65, min_dist=600) ,1
            labell = 'class0: Non pregnant healthy person   class1: other arrhythmia'
    ecg={}

    for w in range (len(DATA)):
        loc = {}
        for i in range (len( DATA[w][1])-1):
            loc[i] = signal.resample((DATA[w][0][DATA[w][1][i]:DATA[w][1][i+1]]), fs) , fs*(fs/(DATA[w][1][i+1]-DATA[w][1][i])) , DATA[w][2]
        ecg[w] = loc

    n_all=0
    for w in range (len(ecg)):
        n_all += len(ecg[w])
    n_train = n_all - len(ecg)
    n_test = len(ecg)
    train_data = np.zeros((n_train,fs))
    test_data =  np.zeros((n_test,fs))
    train_fs = np.zeros(n_train)
    test_fs = np.zeros(n_test)
    train_label = np.zeros(n_train)
    test_label = np.zeros(n_test)


    z=0
    c=0
    for w in range (len(ecg)):
        for i in range (len(ecg[w])-1):
            train_data[z] = ecg[w][i][0]
            train_fs[z]   = ecg[w][i][1]
            train_label[z]= ecg[w][i][2]
            z +=1
        test_data[c] = ecg[w][i][0]
        test_fs[c]   = ecg[w][i][1]
        test_label[c]= ecg[w][i][2]
        c +=1

    train_data , train_fs , train_label = shuffle(train_data , train_fs , train_label)
    test_data , test_fs , test_label = shuffle(test_data , test_fs , test_label)
    total_data = np.vstack((train_data,test_data))
    total_label = np.hstack((train_label,test_label))
    total_data = sklearn.preprocessing.normalize(total_data)


    from sklearn.model_selection import KFold

    kf = KFold(n_splits=10)
    print(kf)
    acc1 =0
    acc2 =0
    for train_index, test_index in kf.split(total_data):
    #     print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = total_data[train_index], total_data[test_index]
        y_train, y_test = total_label[train_index], total_label[test_index]
        pca = PCA(.95)
        pca.fit(X_train)
        X_train= pca.transform(X_train)
        X_test= pca.transform(X_test)
        estimator1= MLPClassifier(hidden_layer_sizes=1000)
        estimator1.fit(X_train, y_train)
        predicted_label1 = estimator1.predict(X_test)
        acc1 += accuracy_score(y_test, predicted_label1)
        estimator2 =sklearn.svm.SVC(kernel='linear')
        estimator2.fit(X_train, y_train)
        predicted_label2 = estimator2.predict(X_test)
        acc2 += accuracy_score(y_test, predicted_label2)
    print('Classification based on :' , labell)
    print('estimator1 (MLP classifier) accuracy : ' , acc1/kf.get_n_splits())
    print('estimator1 (Linear SVM) accuracy : ',  acc2/kf.get_n_splits())
    

C:\Users\Mohamad\Anaconda3\lib\site-packages\peakutils\peak.py:37: RuntimeWarning: overflow encountered in short_scalars
  thres = thres * (np.max(y) - np.min(y)) + np.min(y)


KFold(n_splits=10, random_state=None, shuffle=False)


C:\Users\Mohamad\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Classification based on : class0: Non pregnant healthy person   class1: pregnant person healthy person
estimator1 (MLP classifier) accuracy :  0.846666666667
estimator1 (Linear SVM) accuracy :  0.648888888889
KFold(n_splits=10, random_state=None, shuffle=False)


C:\Users\Mohamad\Anaconda3\lib\site-packages\peakutils\peak.py:37: RuntimeWarning: overflow encountered in short_scalars
  thres = thres * (np.max(y) - np.min(y)) + np.min(y)
C:\Users\Mohamad\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Classification based on : class0: other arrhythmia   class1: pregnant healthy person
estimator1 (MLP classifier) accuracy :  0.977777777778
estimator1 (Linear SVM) accuracy :  0.955555555556
KFold(n_splits=10, random_state=None, shuffle=False)


C:\Users\Mohamad\Anaconda3\lib\site-packages\peakutils\peak.py:37: RuntimeWarning: overflow encountered in short_scalars
  thres = thres * (np.max(y) - np.min(y)) + np.min(y)
C:\Users\Mohamad\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


Classification based on : class0: Non pregnant healthy person   class1: other arrhythmia
estimator1 (MLP classifier) accuracy :  0.977777777778
estimator1 (Linear SVM) accuracy :  0.955555555556


In [12]:
display(pd.DataFrame(confusion_matrix(y_test , predicted_label1)))
print('confusion matrix for MLP by K=10 ')

,0,1
0,4,0
1,0,4


confusion matrix for MLP by K=10 
